In [5]:
# Calculate Performance



In [6]:

import pandas as pd
import time
import nasdaqdatalink
import sys
from datetime import datetime, timedelta
import numpy as np
import sys
sys.path.append('/Users/markwindsor/Desktop/atlas_trade')
from src.utils.performance_metrics import *

from datetime import datetime

today = datetime.today()
five_years_ago = today - timedelta(days=1825)
formatted_date = five_years_ago.strftime('%Y-%m-%d')

import os
from dotenv import load_dotenv
NASDAQ_KEY = os.getenv("NASDAQ_KEY")

nasdaqdatalink.ApiConfig.api_key = NASDAQ_KEY

In [7]:
# Use your absolute path to our top 500 coins csv
universe_path = '/Users/markwindsor/Desktop/atlas_trade/src/data/universe.csv'

universe_df = pd.read_csv(universe_path)

result = []

for index, row in universe_df.iterrows():

    name = row['name']
    ticker = row['ticker']

    row_values = {
        'ticker': ticker,
        'company_name': name
    }

    metrics = nasdaqdatalink.get_table('SHARADAR/METRICS',  qopts={"columns":['ticker', 'beta5y']}, ticker=ticker)
    
    prices = nasdaqdatalink.get_table('SHARADAR/SEP', qopts={"columns":['date', 'close']}, date={'gte':formatted_date}, ticker=ticker)
    prices_df = pd.DataFrame(prices[::-1])
    prices_df["returns"] = np.log(prices_df.close / prices_df.close.shift(1))


    # If we only want to get comanies who has more than 5 year history
    if len(metrics['beta5y'].values):
        beta = metrics['beta5y'].values[0]
        if beta:
            row_values['beta'] = beta
        
            md = max_drawdown(prices_df)
            sharpe = sharpe_ratio(prices_df)
            sortino = sortino_ratio(prices_df)
            treynor = treynor_ratio(prices_df, beta)

            row_values['max_drawdown'] = md
            row_values['sharpe_ratio'] = sharpe
            row_values['sortino_ratio'] = sortino
            row_values['treynor_ratio'] = treynor

    result.append(row_values)


result_df = pd.DataFrame(result)
result_df.to_csv('performanceMetrics.csv', index=False)




